# Chapter 5: Modules and Interfaces

## Module Basics

Modules are the fundamental building blocks of SystemVerilog designs. They encapsulate functionality and provide a way to create hierarchical designs through instantiation.

### Basic Module Structure

```systemverilog
module module_name #(
    // Parameters (optional)
    parameter int WIDTH = 8
) (
    // Port declarations
    input  logic clk,
    input  logic reset_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

    // Module body - internal logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n)
            data_out <= '0;
        else
            data_out <= data_in;
    end

endmodule
```

### Module Instantiation

```systemverilog
// Named port connections (recommended)
module_name #(.WIDTH(16)) inst_name (
    .clk(system_clk),
    .reset_n(sys_reset),
    .data_in(input_data),
    .data_out(output_data)
);

// Positional port connections (not recommended for complex modules)
module_name #(16) inst_name (system_clk, sys_reset, input_data, output_data);
```

### Key Module Concepts

**Scope and Hierarchy**: Each module creates its own scope. Internal signals and variables are not accessible from outside the module unless explicitly connected through ports.

**Instance vs Module**: A module is the template/definition, while an instance is a specific instantiation of that module in your design.

### Example 1: Simple Counter
simple_counter - Basic module structure with parameters and ANSI-style ports

#### Design under Test (DUT) and Testbench

In [3]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_5_examples/example_1__simple_counter/"
files = ["simple_counter.sv", "simple_counter_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_counter.sv
module simple_counter #(
    // Parameters with default values
    parameter int WIDTH = 8,                    // Counter width in bits
    parameter int MAX_COUNT = 255,              // Maximum count value
    parameter bit WRAP_AROUND = 1'b1,           // Enable wrap-around behavior
    parameter int RESET_VALUE = 0               // Reset value for counter
) (
    // ANSI-style port declarations
    input  logic                    clk,        // Clock input
    input  logic                    reset_n,    // Active-low asynchronous reset
    input  logic                    enable,     // Counter enable
    input  logic                    load,       // Load enable
    input  logic [WIDTH-1:0]        load_value, // Value to load
    input  logic                    count_up,   // Count direction (1=up, 0=down)
    output logic [WIDTH-1:0]        count,      // Current counter value
    output logic                    overflow,   // Overflow flag
    output logic                    underflow,  // Underflow flag
    output logic                    max_reached // Maximum count reached flag
);

    // Internal register to hold counter value
    logic [WIDTH-1:0] count_reg;
    
    // Derived parameters using localparam
    localparam logic [WIDTH-1:0] MIN_COUNT = '0;
    localparam logic [WIDTH-1:0] MAX_VAL = MAX_COUNT[WIDTH-1:0];
    localparam logic [WIDTH-1:0] RESET_VAL = RESET_VALUE[WIDTH-1:0];
    
    // Main counter logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            // Asynchronous reset
            count_reg <= RESET_VAL;
            overflow <= 1'b0;
            underflow <= 1'b0;
        end else begin
            // Clear flags by default
            overflow <= 1'b0;
            underflow <= 1'b0;
            
            if (load) begin
                // Load operation has highest priority
                count_reg <= load_value;
            end else if (enable) begin
                if (count_up) begin
                    // Count up logic
                    if (count_reg >= MAX_VAL) begin
                        overflow <= 1'b1;
                        if (WRAP_AROUND) begin
                            count_reg <= MIN_COUNT;
                        end else begin
                            count_reg <= MAX_VAL; // Saturate at maximum
                        end
                    end else begin
                        count_reg <= count_reg + 1'b1;
                    end
                end else begin
                    // Count down logic
                    if (count_reg <= MIN_COUNT) begin
                        underflow <= 1'b1;
                        if (WRAP_AROUND) begin
                            count_reg <= MAX_VAL;
                        end else begin
                            count_reg <= MIN_COUNT; // Saturate at minimum
                        end
                    end else begin
                        count_reg <= count_reg - 1'b1;
                    end
                end
            end
            // If enable is false, counter holds its current value
        end
    end
    
    // Continuous assignments for outputs
    assign count = count_reg;
    assign max_reached = (count_reg == MAX_VAL);
    
    // Assertions for parameter validation (synthesis will ignore these)
    initial begin
        assert (WIDTH > 0)
            else $error("WIDTH parameter must be greater than 0");
        assert (MAX_COUNT >= 0)
            else $error("MAX_COUNT parameter must be non-negative");
        assert (MAX_COUNT < (2**WIDTH))
            else $error("MAX_COUNT exceeds maximum value for given WIDTH");
        assert (RESET_VALUE >= 0)
            else $error("RESET_VALUE parameter must be non-negative");
        assert (RESET_VALUE < (2**WIDTH))
            else $error("RESET_VALUE exceeds maximum value for given WIDTH");
        
        $display();
        $display("Simple Counter Module Initialized:");
        $display("  WIDTH = %0d bits", WIDTH);
        $display("  MAX_COUNT = %0d", MAX_COUNT);
        $display("  WRAP_AROUND = %b", WRAP_AROUND);
        $display("  RESET_VALUE = %0d", RESET_VALUE);
        $display("  Counter range: %0d to %0d", MIN_COUNT, MAX_VAL);
    end

endmodule
```

```systemverilog
// simple_counter_testbench.sv
module simple_counter_testbench;

    // Testbench parameters
    localparam int CLK_PERIOD = 10;  // 100MHz clock
    localparam int TEST_WIDTH = 4;   // 4-bit counter for easy testing
    localparam int TEST_MAX = 12;    // Max count less than 2^4-1 for testing
    
    // Testbench signals
    logic                    clk;
    logic                    reset_n;
    logic                    enable;
    logic                    load;
    logic [TEST_WIDTH-1:0]   load_value;
    logic                    count_up;
    logic [TEST_WIDTH-1:0]   count;
    logic                    overflow;
    logic                    underflow;
    logic                    max_reached;
    
    // Clock generation
    initial begin
        clk = 0;
        forever #(CLK_PERIOD/2) clk = ~clk;
    end
    
    // DUT instantiation with custom parameters
    simple_counter #(
        .WIDTH(TEST_WIDTH),
        .MAX_COUNT(TEST_MAX),
        .WRAP_AROUND(1'b1),
        .RESET_VALUE(0)
    ) dut (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .load(load),
        .load_value(load_value),
        .count_up(count_up),
        .count(count),
        .overflow(overflow),
        .underflow(underflow),
        .max_reached(max_reached)
    );
    
    // Second instance with different parameters for comparison
    logic [7:0] count2;
    logic overflow2, underflow2, max_reached2;
    
    simple_counter #(
        .WIDTH(8),
        .MAX_COUNT(255),
        .WRAP_AROUND(1'b0),     // No wrap-around (saturation mode)
        .RESET_VALUE(128)       // Different reset value
    ) dut2 (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .load(1'b0),            // Disable load for this instance
        .load_value(8'h00),
        .count_up(count_up),
        .count(count2),
        .overflow(overflow2),
        .underflow(underflow2),
        .max_reached(max_reached2)
    );
    
    // Test stimulus
    initial begin
        // Initialize VCD dump
        $dumpfile("simple_counter_testbench.vcd");
        $dumpvars(0, simple_counter_testbench);
        
        $display();
        $display("=== Simple Counter Testbench Started ===");
        $display();
        
        // Initialize signals
        reset_n = 0;
        enable = 0;
        load = 0;
        load_value = 0;
        count_up = 1;
        
        // Reset phase
        $display("Phase 1: Reset Test");
        #(CLK_PERIOD * 2);
        reset_n = 1;
        #(CLK_PERIOD);
        $display("After reset - DUT1 count: %0d", count);
        $display("After reset - DUT2 count: %0d", count2);
        $display();
        
        // Test counting up
        $display("Phase 2: Count Up Test");
        enable = 1;
        count_up = 1;
        
        repeat (TEST_MAX + 3) begin
            #(CLK_PERIOD);
            $display(
                "Count up - DUT1: %0d (overflow=%b, max_reached=%b)", 
                count, overflow, max_reached);
            $display("Count up - DUT2: %0d (overflow=%b)", count2, overflow2);
        end
        $display();
        
        // Test counting down
        $display("Phase 3: Count Down Test");
        count_up = 0;
        
        repeat (TEST_MAX + 3) begin
            #(CLK_PERIOD);
            $display(
                "Count down - DUT1: %0d (underflow=%b)", count, underflow);
            $display(
                "Count down - DUT2: %0d (underflow=%b)", count2, underflow2);
        end
        $display();
        
        // Test load operation
        $display("Phase 4: Load Operation Test");
        count_up = 1;
        load_value = TEST_WIDTH'(TEST_MAX / 2);  // Explicit width casting
        load = 1;
        #(CLK_PERIOD);
        $display("After load %0d - DUT1 count: %0d", load_value, count);
        load = 0;
        #(CLK_PERIOD);
        $display("After load release - DUT1 count: %0d", count);
        $display();
        
        // Test enable control
        $display("Phase 5: Enable Control Test");
        enable = 0;
        repeat (3) begin
            #(CLK_PERIOD);
            $display("Enable=0 - DUT1 count: %0d (should not change)", count);
        end
        
        enable = 1;
        repeat (3) begin
            #(CLK_PERIOD);
            $display("Enable=1 - DUT1 count: %0d (should increment)", count);
        end
        $display();
        
        // Test different parameter behavior
        $display("Phase 6: Parameter Comparison");
        $display(
            "DUT1 (4-bit, wrap-around): count=%0d, max_count=%0d",
            count, TEST_MAX);
        $display(
            "DUT2 (8-bit, saturation): count=%0d, max_count=255",
            count2);
        $display();
        
        // Final phase - reset test
        $display("Phase 7: Final Reset Test");
        reset_n = 0;
        #(CLK_PERIOD);
        reset_n = 1;
        #(CLK_PERIOD);
        $display("After final reset - DUT1: %0d, DUT2: %0d", count, count2);
        $display();
        
        $display("=== Simple Counter Testbench Completed ===");
        $display("Total simulation time: %0t", $time);
        $display();
        $finish;
    end
    
    // Monitor for detecting important events
    always @(posedge clk) begin
        if (reset_n) begin
            if (overflow)
                $display(
                    "*** OVERFLOW detected at time %0t, count=%0d ***",
                    $time, count);
            if (underflow)
                $display(
                    "*** UNDERFLOW detected at time %0t, count=%0d ***",
                    $time, count);
            if (max_reached && enable)
                $display(
                    "*** MAX_REACHED at time %0t, count=%0d ***",
                    $time, count);
        end
    end

endmodule
```

Docker Compose Output:

Simple Counter Module Initialized:
  WIDTH = 4 bits
  MAX_COUNT = 12
  WRAP_AROUND = 1
  RESET_VALUE = 0
  Counter range: 0 to 12

Simple Counter Module Initialized:
  WIDTH = 8 bits
  MAX_COUNT = 255
  WRAP_AROUND = 0
  RESET_VALUE = 128
  Counter range: 0 to 255

=== Simple Counter Testbench Started ===

Phase 1: Reset Test
After reset - DUT1 count: 0
After reset - DUT2 count: 128

Phase 2: Count Up Test
Count up - DUT1: 1 (overflow=0, max_reached=0)
Count up - DUT2: 129 (overflow=0)
Count up - DUT1: 2 (overflow=0, max_reached=0)
Count up - DUT2: 130 (overflow=0)
Count up - DUT1: 3 (overflow=0, max_reached=0)
Count up - DUT2: 131 (overflow=0)
Count up - DUT1: 4 (overflow=0, max_reached=0)
Count up - DUT2: 132 (overflow=0)
Count up - DUT1: 5 (overflow=0, max_reached=0)
Count up - DUT2: 133 (overflow=0)
Count up - DUT1: 6 (overflow=0, max_reached=0)
Count up - DUT2: 134 (overflow=0)
Count up - DUT1: 7 (overflow=0, max_reached=0)
Count up - DUT2: 135 (overflow=0)

0

In [2]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_1__simple_counter/")


Docker Compose Output:
 Container notebooks-verilator-1  Creating
 Container notebooks-verilator-1  Created
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started
Could not initialize GTK!  Is DISPLAY env var/xhost set?

Usage: gtkwave [OPTION]... [DUMPFILE] [SAVEFILE] [RCFILE]

  -n, --nocli=DIRPATH        use file requester for dumpfile name
  -f, --dump=FILE            specify dumpfile name
  -F, --fastload             generate/use VCD recoder fastload files
  -o, --optimize             optimize VCD to FST
  -a, --save=FILE            specify savefile name
  -A, --autosavename         assume savefile is suffix modified dumpfile name
  -r, --rcfile=FILE          specify override .rcfile name
  -d, --defaultskip          if missing .rcfile, do not use useful defaults
  -D, --dualid=WHICH         specify multisession identifier
  -l, --logfile=FILE         specify simulation logfile name for time values
  -s, --start=TIME           specify start time for L

0

### Example 2: Data Register
data_register - Module instantiation examples (named vs positional connections)

#### Design under Test (DUT) and Testbench

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_5_examples/example_2__data_register/"
files = ["data_register.sv", "data_register_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_register.sv
module data_register (
    input  logic       clk,     // Clock signal
    input  logic       rst_n,   // Active-low reset
    input  logic       enable,  // Enable signal
    input  logic [7:0] data_in, // 8-bit input data
    output logic [7:0] data_out // 8-bit output data
);

    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            data_out <= 8'h00;  // Reset to zero
        end else if (enable) begin
            data_out <= data_in; // Load input data when enabled
        end
        // If enable is low, data_out retains its value
    end

    // Display messages for simulation
    initial begin
        $display();
        $display("Data Register module initialized");
        $display("  - 8-bit register with enable control");
        $display("  - Active-low reset");
        $display();
    end

endmodule
```

```systemverilog
// data_register_testbench.sv
module data_register_testbench;

    // Testbench signals
    logic       tb_clk;
    logic       tb_rst_n;
    logic       tb_enable;
    logic [7:0] tb_data_in;
    logic [7:0] tb_data_out_named;
    logic [7:0] tb_data_out_positional;

    // Clock generation
    initial begin
        tb_clk = 0;
        forever #5 tb_clk = ~tb_clk; // 10ns period clock
    end

    // =================================================================
    // EXAMPLE 1: NAMED PORT CONNECTIONS (Recommended method)
    // =================================================================
    data_register DUT_NAMED_CONNECTIONS (
        .clk(tb_clk),
        .rst_n(tb_rst_n),
        .enable(tb_enable),
        .data_in(tb_data_in),
        .data_out(tb_data_out_named)
    );

    // =================================================================
    // EXAMPLE 2: POSITIONAL PORT CONNECTIONS (Order matters!)
    // =================================================================
    data_register DUT_POSITIONAL_CONNECTIONS (
        tb_clk,                    // clk (1st port)
        tb_rst_n,                  // rst_n (2nd port)
        tb_enable,                 // enable (3rd port)
        tb_data_in,                // data_in (4th port)
        tb_data_out_positional     // data_out (5th port)
    );

    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("data_register_testbench.vcd");
        $dumpvars(0, data_register_testbench);

        $display("=== Data Register Testbench Started ===");
        $display("Testing both named and positional instantiation methods");
        $display();

        // Initialize signals
        tb_rst_n = 0;
        tb_enable = 0;
        tb_data_in = 8'h00;

        // Reset sequence
        $display("Time %0t: Applying reset", $time);
        #20;
        tb_rst_n = 1;
        $display("Time %0t: Releasing reset", $time);
        #10;

        // Test 1: Load data with enable
        tb_enable = 1;
        tb_data_in = 8'hAA;
        $display("Time %0t: Loading data 0x%02h with enable=1", $time, tb_data_in);
        #20;
        $display("Time %0t: Named output = 0x%02h, Positional output = 0x%02h",
                 $time, tb_data_out_named, tb_data_out_positional);

        // Test 2: Change input with enable disabled
        tb_enable = 0;
        tb_data_in = 8'h55;
        $display("Time %0t: Changing input to 0x%02h with enable=0",
                 $time, tb_data_in);
        #20;
        $display("Time %0t: Named output = 0x%02h, Positional output = 0x%02h",
                 $time, tb_data_out_named, tb_data_out_positional);
        $display("  -> Data should remain unchanged (0xAA)");

        // Test 3: Enable again to load new data
        tb_enable = 1;
        $display("Time %0t: Re-enabling to load new data 0x%02h",
                 $time, tb_data_in);
        #20;
        $display("Time %0t: Named output = 0x%02h, Positional output = 0x%02h",
                 $time, tb_data_out_named, tb_data_out_positional);

        $display();
        $display("=== Both instantiation methods produce identical results ===");
        $display("Named connections: More readable and less error-prone");
        $display("Positional connections: Shorter but order-dependent");
        $display();
        
        #10;
        $finish;
    end

endmodule
```

Docker Compose Output:

Data Register module initialized
  - 8-bit register with enable control
  - Active-low reset


Data Register module initialized
  - 8-bit register with enable control
  - Active-low reset

=== Data Register Testbench Started ===
Testing both named and positional instantiation methods

Time 0: Applying reset
Time 20: Releasing reset
Time 30: Loading data 0xaa with enable=1
Time 50: Named output = 0xaa, Positional output = 0xaa
Time 50: Changing input to 0x55 with enable=0
Time 70: Named output = 0xaa, Positional output = 0xaa
  -> Data should remain unchanged (0xAA)
Time 70: Re-enabling to load new data 0x55
Time 90: Named output = 0x55, Positional output = 0x55

=== Both instantiation methods produce identical results ===
Named connections: More readable and less error-prone
Positional connections: Shorter but order-dependent

Process finished with return code: 0
Removing Chapter_5_examples/example_2__data_register/obj_dir directory...
Chapter_5_examples/example

0

## Port Declarations and Directions

SystemVerilog provides several ways to declare module ports, offering more flexibility than traditional Verilog.

### Port Directions

```systemverilog
module port_example (
    input  logic        clk,           // Input port
    output logic        valid,         // Output port
    inout  wire         bidir_signal,  // Bidirectional port
    ref    int          shared_var     // Reference port (SystemVerilog)
);
```

### ANSI-Style Port Declarations (Recommended)

```systemverilog
module counter #(
    parameter int WIDTH = 8
) (
    input  logic             clk,
    input  logic             reset_n,
    input  logic             enable,
    input  logic             load,
    input  logic [WIDTH-1:0] load_value,
    output logic [WIDTH-1:0] count,
    output logic             overflow
);

    logic [WIDTH-1:0] count_reg;
    
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            count_reg <= '0;
            overflow <= 1'b0;
        end else if (load) begin
            count_reg <= load_value;
            overflow <= 1'b0;
        end else if (enable) begin
            {overflow, count_reg} <= count_reg + 1'b1;
        end
    end
    
    assign count = count_reg;

endmodule
```

### Non-ANSI Style (Legacy)

```systemverilog
module counter (clk, reset_n, enable, count);
    parameter WIDTH = 8;
    
    input             clk;
    input             reset_n;
    input             enable;
    output [WIDTH-1:0] count;
    
    // Port declarations separate from module header
endmodule
```

### Advanced Port Features

**Interface Ports**:
```systemverilog
module processor (
    input logic clk,
    input logic reset_n,
    memory_if.master mem_bus,  // Interface port
    axi4_if.slave    axi_port
);
```

**Unpacked Array Ports**:
```systemverilog
module multi_port (
    input  logic [7:0] data_in [0:3],   // Array of inputs
    output logic [7:0] data_out [0:3]   // Array of outputs
);
```

### Example 3: Port Direction
port_direction - Different port types (input, output, inout, ref)

In [12]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_5_examples/example_3__port_direction/"
files = ["port_direction.sv", "port_direction_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// port_direction.sv
module port_direction (
    // INPUT ports - data flows INTO the module
    input  logic       clk,        // Clock signal
    input  logic       reset_n,    // Active-low reset
    input  logic [3:0] data_in,    // 4-bit input data
    
    // OUTPUT ports - data flows OUT of the module
    output logic [3:0] data_out,   // 4-bit output data
    output logic       valid_out,  // Output valid flag
    
    // INOUT ports - bidirectional data flow
    inout  wire  [7:0] bus_data,   // 8-bit bidirectional bus
    
    // REF ports - pass by reference (SystemVerilog only)
    ref    logic [1:0] ref_counter // Reference to external counter
);

    // Internal signals
    logic       bus_enable;    // Controls bus direction
    logic [7:0] internal_bus;  // Internal bus data
    
    // Simple counter for demonstration
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            data_out <= 4'h0;
            valid_out <= 1'b0;
            internal_bus <= 8'h00;
            bus_enable <= 1'b0;
        end else begin
            // Process input data (double it)
            data_out <= data_in * 2;
            valid_out <= |data_in;  // Valid if any input bit is set
            
            // Bus control logic
            bus_enable <= data_in[0];  // Use LSB to control bus
            if (data_in[0]) begin
                internal_bus <= {4'h0, data_in};  // Drive bus
            end
            
            // Modify ref_counter directly (demonstrates ref port)
            if (valid_out) begin
                ref_counter <= ref_counter + 1;
            end
        end
    end
    
    // Bidirectional bus driver
    assign bus_data = bus_enable ? internal_bus : 8'hZZ;
    
    // Display messages for simulation
    initial begin
        $display();
        $display("Port Direction Demo module initialized");
        $display("  - INPUT: clk, reset_n, data_in");
        $display("  - OUTPUT: data_out, valid_out");
        $display("  - INOUT: bus_data (bidirectional)");
        $display("  - REF: ref_counter (pass by reference)");
        $display();
    end

endmodule
```

```systemverilog
// port_direction_testbench.sv
module port_direction_testbench;

    // Testbench signals for different port types
    logic       tb_clk;
    logic       tb_reset_n;
    logic [3:0] tb_data_in;        // INPUT port connection
    logic [3:0] tb_data_out;       // OUTPUT port connection
    logic       tb_valid_out;      // OUTPUT port connection
    wire  [7:0] tb_bus_data;       // INOUT port connection (wire)
    logic [1:0] tb_ref_counter;    // REF port connection
    
    // Additional signals for testing inout port
    logic       tb_bus_drive;
    logic [7:0] tb_bus_value;

    // Clock generation
    initial begin
        tb_clk = 0;
        forever #5 tb_clk = ~tb_clk; // 10ns period clock
    end

    // =============================================================
    // MODULE INSTANTIATION - Demonstrating all port types
    // =============================================================
    port_direction DUT (
        // INPUT ports - we drive these from testbench
        .clk(tb_clk),
        .reset_n(tb_reset_n),
        .data_in(tb_data_in),
        
        // OUTPUT ports - module drives these to testbench
        .data_out(tb_data_out),
        .valid_out(tb_valid_out),
        
        // INOUT port - bidirectional connection
        .bus_data(tb_bus_data),
        
        // REF port - direct reference to testbench variable
        .ref_counter(tb_ref_counter)
    );

    // Bidirectional bus driver for testing
    assign tb_bus_data = tb_bus_drive ? tb_bus_value : 8'hZZ;

    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("port_direction_testbench.vcd");
        $dumpvars(0, port_direction_testbench);

        $display("=== Port Direction Demo Testbench Started ===");
        $display();

        // Initialize signals
        tb_reset_n = 0;
        tb_data_in = 4'h0;
        tb_ref_counter = 2'b00;
        tb_bus_drive = 0;
        tb_bus_value = 8'h00;

        // Reset sequence
        $display("Time %0t: Applying reset", $time);
        #20;
        tb_reset_n = 1;
        $display("Time %0t: Releasing reset", $time);
        #10;

        // Test 1: INPUT and OUTPUT ports
        $display("--- Testing INPUT and OUTPUT ports ---");
        tb_data_in = 4'h3;
        #20;
        $display("Time %0t: INPUT data_in = %d, OUTPUT data_out = %d, valid = %b", 
                 $time, tb_data_in, tb_data_out, tb_valid_out);
        $display("  -> Module doubles input: %d * 2 = %d", 
                 tb_data_in, tb_data_out);
        
        // Test 2: REF port modification
        $display("--- Testing REF port ---");
        $display("Time %0t: REF counter before = %d", $time, tb_ref_counter);
        tb_data_in = 4'h5;
        #20;
        $display("Time %0t: REF counter after = %d", $time, tb_ref_counter);
        $display("  -> Module modified ref_counter directly!");

        // Test 3: INOUT port - Module driving bus
        $display("--- Testing INOUT port - Module driving ---");
        tb_data_in = 4'h7;  // LSB = 1, so module will drive bus
        #20;
        $display("Time %0t: Module driving bus_data = 0x%02h", 
                 $time, tb_bus_data);
        $display("  -> Module drives bus when data_in[0] = 1");

        // Test 4: INOUT port - Testbench driving bus
        $display("--- Testing INOUT port - Testbench driving ---");
        tb_data_in = 4'h6;  // LSB = 0, so module tri-states bus
        tb_bus_drive = 1;
        tb_bus_value = 8'hFF;
        #20;
        $display("Time %0t: Testbench driving bus_data = 0x%02h", 
                 $time, tb_bus_data);
        $display("  -> Testbench drives bus when module tri-states");

        // Test 5: Show final state
        tb_bus_drive = 0;
        tb_data_in = 4'h0;
        #20;
        $display();
        $display("=== Final Results ===");
        $display("INPUT ports: Driven by testbench to module");
        $display("OUTPUT ports: Driven by module to testbench");
        $display("INOUT ports: Bidirectional, either side can drive");
        $display("REF ports: Direct reference, module can modify testbench vars");
        $display("Final ref_counter value: %d", tb_ref_counter);
        $display();

        #10;
        $finish;
    end

endmodule
```

Docker Compose Output:

Port Direction Demo module initialized
  - INPUT: clk, reset_n, data_in
  - OUTPUT: data_out, valid_out
  - INOUT: bus_data (bidirectional)
  - REF: ref_counter (pass by reference)

=== Port Direction Demo Testbench Started ===

Time 0: Applying reset
Time 20: Releasing reset
--- Testing INPUT and OUTPUT ports ---
Time 50: INPUT data_in =  3, OUTPUT data_out =  6, valid = 1
  -> Module doubles input:  3 * 2 =  6
--- Testing REF port ---
Time 50: REF counter before = 1
Time 70: REF counter after = 3
  -> Module modified ref_counter directly!
--- Testing INOUT port - Module driving ---
Time 90: Module driving bus_data = 0x07
  -> Module drives bus when data_in[0] = 1
--- Testing INOUT port - Testbench driving ---
Time 110: Testbench driving bus_data = 0xff
  -> Testbench drives bus when module tri-states

=== Final Results ===
INPUT ports: Driven by testbench to module
OUTPUT ports: Driven by module to testbench
INOUT ports: Bidirectional, either side can drive
RE

0

### Example 4: Array Port Module
array_port_module - Unpacked array ports demonstration

In [14]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_5_examples/example_4__array_port_module/"
files = ["array_port_module.sv", "array_port_module_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=False)


```systemverilog
// array_port_module.sv - Fixed version with proper bit widths
module array_port_module #(
    parameter int DATA_WIDTH = 8,               // Width of each data element
    parameter int NUM_CHANNELS = 4              // Number of parallel channels
) (
    // Clock and reset
    input  logic                            clk,
    input  logic                            reset_n,
    input  logic                            enable,
    
    // Unpacked array input ports - multiple data channels
    input  logic [DATA_WIDTH-1:0]          data_in [NUM_CHANNELS],     // Array of input data
    input  logic                            valid_in [NUM_CHANNELS],    // Array of valid signals
    
    // Unpacked array output ports
    output logic [DATA_WIDTH-1:0]          data_out [NUM_CHANNELS],    // Array of output data
    output logic                            valid_out [NUM_CHANNELS],   // Array of valid outputs
    
    // Status arrays
    output logic [7:0]                      channel_count [NUM_CHANNELS],   // Per-channel counters
    output logic [DATA_WIDTH-1:0]          channel_max [NUM_CHANNELS],     // Maximum value per channel
    output logic [DATA_WIDTH-1:0]          channel_min [NUM_CHANNELS]      // Minimum value per channel
);

    // Generate block for per-channel processing
    generate
        genvar ch;
        for (ch = 0; ch < NUM_CHANNELS; ch++) begin : channel_proc
            
            // Per-channel registers
            logic [DATA_WIDTH-1:0] data_reg;
            logic                   valid_reg;
            logic [7:0]            counter;
            logic [DATA_WIDTH-1:0] max_val;
            logic [DATA_WIDTH-1:0] min_val;
            
            // Per-channel processing logic
            always_ff @(posedge clk or negedge reset_n) begin
                if (!reset_n) begin
                    data_reg <= '0;
                    valid_reg <= 1'b0;
                    counter <= '0;
                    max_val <= '0;
                    min_val <= '1; // All 1s for minimum initialization
                end else if (enable) begin
                    // Process input data
                    data_reg <= data_in[ch];
                    valid_reg <= valid_in[ch];
                    
                    // Update statistics when valid data arrives
                    if (valid_in[ch]) begin
                        counter <= counter + 1'b1;
                        
                        // Track max/min values
                        if (data_in[ch] > max_val) begin
                            max_val <= data_in[ch];
                        end
                        if (data_in[ch] < min_val) begin
                            min_val <= data_in[ch];
                        end
                    end
                end
            end
            
            // Connect internal registers to output arrays
            assign data_out[ch] = data_reg;
            assign valid_out[ch] = valid_reg;
            assign channel_count[ch] = counter;
            assign channel_max[ch] = max_val;
            assign channel_min[ch] = min_val;
        end
    endgenerate
    
    // Cross-channel operations using array manipulation
    logic [DATA_WIDTH-1:0] sum_all_channels;
    logic [$clog2(NUM_CHANNELS+1)-1:0] active_channels;
    
    // Calculate sum and count of active channels
    always_comb begin
        sum_all_channels = '0;
        active_channels = '0;
        
        for (int i = 0; i < NUM_CHANNELS; i++) begin
            if (valid_out[i]) begin
                sum_all_channels += data_out[i];
                active_channels++;
            end
        end
    end
    
    // Simple array comparison example - fixed width truncation
    logic [DATA_WIDTH-1:0] highest_value;
    logic [$clog2(NUM_CHANNELS)-1:0] highest_channel;
    
    always_comb begin
        highest_value = '0;
        highest_channel = '0;
        
        for (int i = 0; i < NUM_CHANNELS; i++) begin
            if (valid_out[i] && (data_out[i] > highest_value)) begin
                highest_value = data_out[i];
                highest_channel = i[$clog2(NUM_CHANNELS)-1:0]; // Fixed: proper width conversion
            end
        end
    end
    
    // Parameter validation and info display
    initial begin
        assert (NUM_CHANNELS > 0 && NUM_CHANNELS <= 16) 
            else $error("NUM_CHANNELS must be between 1 and 16");
        assert (DATA_WIDTH > 0 && DATA_WIDTH <= 16) 
            else $error("DATA_WIDTH must be between 1 and 16");
        
        $display("Simple Array Port Module Initialized:");
        $display("  DATA_WIDTH = %0d bits", DATA_WIDTH);
        $display("  NUM_CHANNELS = %0d", NUM_CHANNELS);
    end

endmodule
```

```systemverilog
// array_port_module_testbench.sv - Fixed testbench with proper array syntax
module array_port_module_testbench;

    // Testbench parameters
    localparam int CLK_PERIOD = 10;
    localparam int TB_DATA_WIDTH = 8;
    localparam int TB_NUM_CHANNELS = 4;
    
    // Testbench signals - using arrays to match DUT
    logic                               clk;
    logic                               reset_n;
    logic                               enable;
    
    // Array signals
    logic [TB_DATA_WIDTH-1:0]          data_in [TB_NUM_CHANNELS];
    logic                               valid_in [TB_NUM_CHANNELS];
    logic [TB_DATA_WIDTH-1:0]          data_out [TB_NUM_CHANNELS];
    logic                               valid_out [TB_NUM_CHANNELS];
    logic [7:0]                        channel_count [TB_NUM_CHANNELS];
    logic [TB_DATA_WIDTH-1:0]          channel_max [TB_NUM_CHANNELS];
    logic [TB_DATA_WIDTH-1:0]          channel_min [TB_NUM_CHANNELS];
    
    // Clock generation
    initial begin
        clk = 0;
        forever #(CLK_PERIOD/2) clk = ~clk;
    end
    
    // DUT instantiation
    array_port_module #(
        .DATA_WIDTH(TB_DATA_WIDTH),
        .NUM_CHANNELS(TB_NUM_CHANNELS)
    ) dut (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .data_in(data_in),
        .valid_in(valid_in),
        .data_out(data_out),
        .valid_out(valid_out),
        .channel_count(channel_count),
        .channel_max(channel_max),
        .channel_min(channel_min)
    );
    
    // Task to send data to a specific channel
    task automatic send_data(input int channel, input logic [TB_DATA_WIDTH-1:0] data, input logic valid);
        if (channel < TB_NUM_CHANNELS) begin
            data_in[channel] = data;
            valid_in[channel] = valid;
        end
    endtask
    
    // Task to display all channel status
    task automatic display_status();
        $display("\n=== Channel Status at time %0t ===", $time);
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            $display("Channel %0d: out=%3d, valid=%b, count=%3d, max=%3d, min=%3d",
                    i, data_out[i], valid_out[i], channel_count[i], 
                    channel_max[i], channel_min[i]);
        end
        $display("====================================\n");
    endtask
    
    // Test stimulus
    initial begin
        // Initialize VCD dump
        $dumpfile("array_port_module_testbench.vcd");
        $dumpvars(0, array_port_module_testbench);
        
        $display("=== Simple Array Port Module Testbench Started ===");
        $display("Testing with %0d channels, %0d-bit data width", TB_NUM_CHANNELS, TB_DATA_WIDTH);
        
        // Initialize all signals
        reset_n = 0;
        enable = 0;
        
        // Initialize input arrays
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            data_in[i] = 0;
            valid_in[i] = 0;
        end
        
        // Reset sequence
        $display("\nPhase 1: Reset Test");
        #(CLK_PERIOD * 2);
        reset_n = 1;
        enable = 1;
        #(CLK_PERIOD);
        display_status();
        
        // Phase 2: Send data to all channels simultaneously
        $display("Phase 2: Send Data to All Channels");
        for (int cycle = 0; cycle < 5; cycle++) begin
            $display("Cycle %0d:", cycle);
            for (int ch = 0; ch < TB_NUM_CHANNELS; ch++) begin
                send_data(ch, 8'(cycle * 20 + ch * 5), 1'b1);
            end
            #(CLK_PERIOD);
            display_status();
        end
        
        // Phase 3: Send data to specific channels only
        $display("Phase 3: Selective Channel Operation");
        
        // Clear all valid signals first
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            valid_in[i] = 0;
        end
        #(CLK_PERIOD);
        
        // Send to channel 0 and 2 only
        send_data(0, 8'd150, 1'b1);
        send_data(2, 8'd200, 1'b1);
        #(CLK_PERIOD);
        display_status();
        
        // Send to channel 1 and 3 only
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            valid_in[i] = 0;
        end
        send_data(1, 8'd75, 1'b1);
        send_data(3, 8'd250, 1'b1);
        #(CLK_PERIOD);
        display_status();
        
        // Phase 4: Test min/max tracking
        $display("Phase 4: Min/Max Value Tracking");
        
        // Use the pre-declared test_val variable
        for (int cycle = 0; cycle < 5; cycle++) begin
            case (cycle)
                0: test_val = 50;
                1: test_val = 200;
                2: test_val = 10;
                3: test_val = 240;
                4: test_val = 30;
                default: test_val = 0;
            endcase
            
            $display("Sending test value %0d to all channels", test_val);
            for (int ch = 0; ch < TB_NUM_CHANNELS; ch++) begin
                send_data(ch, test_val, 1'b1);
            end
            #(CLK_PERIOD);
            display_status();
        end
        
        // Phase 5: Test enable control
        $display("Phase 5: Enable Control Test");
        
        // Disable the module
        enable = 0;
        for (int ch = 0; ch < TB_NUM_CHANNELS; ch++) begin
            send_data(ch, 8'd100, 1'b1);
        end
        #(CLK_PERIOD * 2);
        $display("With enable=0 (should not process new data):");
        display_status();
        
        // Re-enable
        enable = 1;
        #(CLK_PERIOD);
        $display("With enable=1 (should process new data):");
        display_status();
        
        // Phase 6: Counter test
        $display("Phase 6: Counter Test - Send Multiple Values");
        for (int burst = 0; burst < 3; burst++) begin
            for (int ch = 0; ch < TB_NUM_CHANNELS; ch++) begin
                send_data(ch, 8'(burst * 50 + ch * 10), 1'b1);
            end
            #(CLK_PERIOD);
        end
        display_status();
        
        // Phase 7: Final test with mixed valid signals
        $display("Phase 7: Mixed Valid Signals Test");
        for (int cycle = 0; cycle < 4; cycle++) begin
            for (int ch = 0; ch < TB_NUM_CHANNELS; ch++) begin
                // Alternate valid signals in a pattern - fixed width truncation
                logic valid = ((cycle + ch) % 2) == 1;
                send_data(ch, 8'(cycle * 30 + ch), valid);
            end
            #(CLK_PERIOD);
            if (cycle % 2 == 1) display_status();
        end
        
        // Final status
        $display("Final Status:");
        // Clear all inputs
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            valid_in[i] = 0;
        end
        #(CLK_PERIOD);
        display_status();
        
        $display("=== Array Port Module Testbench Completed ===");
        $display("Total simulation time: %0t", $time);
        $finish;
    end
    
    // Monitor for interesting events
    always @(posedge clk) begin
        if (reset_n && enable) begin
            // Count how many channels are active
            int active_count = 0;
            for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
                if (valid_in[i]) active_count++;
            end
            
            if (active_count == TB_NUM_CHANNELS) begin
                $display("*** ALL %0d CHANNELS ACTIVE at time %0t ***", TB_NUM_CHANNELS, $time);
            end
        end
    end
    
    // Example of array manipulation in testbench
    logic [TB_DATA_WIDTH-1:0] sum_outputs;
    logic [TB_DATA_WIDTH-1:0] max_output;
    int valid_output_count;
    
    // Test value variable for Phase 4
    logic [TB_DATA_WIDTH-1:0] test_val;
    
    // Calculate statistics from output arrays
    always_comb begin
        sum_outputs = '0;
        max_output = '0;
        valid_output_count = 0;
        
        for (int i = 0; i < TB_NUM_CHANNELS; i++) begin
            if (valid_out[i]) begin
                sum_outputs += data_out[i];
                valid_output_count++;
                if (data_out[i] > max_output) begin
                    max_output = data_out[i];
                end
            end
        end
    end

endmodule
```

Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_4__array_port_module/")


In [ ]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_4__array_port_module/")


## Parameters and Localparams

Parameters provide a way to create configurable, reusable modules. They allow customization at instantiation time.

### Parameter Types

```systemverilog
module parameterized_module #(
    // Type parameters
    parameter type DATA_TYPE = logic [31:0],
    parameter type ADDR_TYPE = logic [15:0],
    
    // Value parameters
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16,
    parameter int DEPTH = 1024,
    
    // String parameters
    parameter string MODE = "NORMAL",
    
    // Real parameters
    parameter real FREQUENCY = 100.0
) (
    input  logic      clk,
    input  DATA_TYPE  data_in,
    input  ADDR_TYPE  address,
    output DATA_TYPE  data_out
);
```

### Localparam Usage

Localparams are parameters that cannot be overridden during instantiation. They're typically used for derived values.

```systemverilog
module memory #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 10
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    // Localparams derived from parameters
    localparam int DEPTH = 2**ADDR_WIDTH;
    localparam int BYTES_PER_WORD = DATA_WIDTH / 8;
    
    logic [DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge clk) begin
        if (we)
            mem_array[addr] <= wdata;
        rdata <= mem_array[addr];
    end

endmodule
```

### Parameter Override Examples

```systemverilog
// Override during instantiation
memory #(
    .DATA_WIDTH(64),
    .ADDR_WIDTH(12)
) ram_inst (
    .clk(clk),
    .we(write_enable),
    .addr(address),
    .wdata(write_data),
    .rdata(read_data)
);

// Using defparam (not recommended)
defparam ram_inst.DATA_WIDTH = 64;
defparam ram_inst.ADDR_WIDTH = 12;
```

### Example 5: Configurable Memory
configurable_memory - Parameter types (type, value, string, real parameters)

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_5__configurable_memory/")


In [ ]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_5__configurable_memory/")


### Example 6: Parameter Override
parameter_override - Parameter override during instantiation

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_6__parameter_override/")


In [ ]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_6__parameter_override/")


### Example 7: Localparam Calculator
localparam_calculator - Localparam usage for derived values

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_7__localparam_calculator/")


## Generate Blocks

Generate blocks allow you to create repetitive hardware structures and conditional compilation based on parameters.

### Generate For Loops

```systemverilog
module parallel_adder #(
    parameter int WIDTH = 32,
    parameter int STAGES = 4
) (
    input  logic [WIDTH-1:0] a,
    input  logic [WIDTH-1:0] b,
    input  logic             cin,
    output logic [WIDTH-1:0] sum,
    output logic             cout
);

    localparam int BITS_PER_STAGE = WIDTH / STAGES;
    
    logic [STAGES:0] carry;
    assign carry[0] = cin;
    assign cout = carry[STAGES];
    
    // Generate multiple adder stages
    generate
        for (genvar i = 0; i < STAGES; i++) begin : adder_stage
            logic [BITS_PER_STAGE-1:0] stage_sum;
            logic                      stage_cout;
            
            full_adder #(.WIDTH(BITS_PER_STAGE)) fa_inst (
                .a(a[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .b(b[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .cin(carry[i]),
                .sum(stage_sum),
                .cout(stage_cout)
            );
            
            assign sum[i*BITS_PER_STAGE +: BITS_PER_STAGE] = stage_sum;
            assign carry[i+1] = stage_cout;
        end
    endgenerate

endmodule
```

### Generate If-Else

```systemverilog
module configurable_memory #(
    parameter int    DATA_WIDTH = 32,
    parameter int    ADDR_WIDTH = 10,
    parameter string MEMORY_TYPE = "BLOCK"  // "BLOCK" or "DISTRIBUTED"
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    localparam int DEPTH = 2**ADDR_WIDTH;
    
    generate
        if (MEMORY_TYPE == "BLOCK") begin : block_memory
            // Use block RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
                rdata <= mem[addr];
            end
            
        end else if (MEMORY_TYPE == "DISTRIBUTED") begin : dist_memory
            // Use distributed RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
            end
            
            assign rdata = mem[addr];  // Combinational read
            
        end else begin : error_memory
            // Generate compile-time error for invalid parameter
            initial begin
                $error("Invalid MEMORY_TYPE parameter: %s", MEMORY_TYPE);
            end
        end
    endgenerate

endmodule
```

### Generate Case

```systemverilog
module priority_encoder #(
    parameter int WIDTH = 8
) (
    input  logic [WIDTH-1:0] data_in,
    output logic [$clog2(WIDTH)-1:0] encoded_out,
    output logic valid
);

    generate
        case (WIDTH)
            4: begin : enc_4bit
                always_comb begin
                    casez (data_in)
                        4'b???1: {valid, encoded_out} = {1'b1, 2'd0};
                        4'b??10: {valid, encoded_out} = {1'b1, 2'd1};
                        4'b?100: {valid, encoded_out} = {1'b1, 2'd2};
                        4'b1000: {valid, encoded_out} = {1'b1, 2'd3};
                        default: {valid, encoded_out} = {1'b0, 2'd0};
                    endcase
                end
            end
            
            8: begin : enc_8bit
                // Implementation for 8-bit encoder
                always_comb begin
                    casez (data_in)
                        8'b???????1: {valid, encoded_out} = {1'b1, 3'd0};
                        8'b??????10: {valid, encoded_out} = {1'b1, 3'd1};
                        8'b?????100: {valid, encoded_out} = {1'b1, 3'd2};
                        8'b????1000: {valid, encoded_out} = {1'b1, 3'd3};
                        8'b???10000: {valid, encoded_out} = {1'b1, 3'd4};
                        8'b??100000: {valid, encoded_out} = {1'b1, 3'd5};
                        8'b?1000000: {valid, encoded_out} = {1'b1, 3'd6};
                        8'b10000000: {valid, encoded_out} = {1'b1, 3'd7};
                        default:     {valid, encoded_out} = {1'b0, 3'd0};
                    endcase
                end
            end
            
            default: begin : enc_generic
                // Generic implementation for other widths
                always_comb begin
                    encoded_out = '0;
                    valid = 1'b0;
                    for (int i = 0; i < WIDTH; i++) begin
                        if (data_in[i]) begin
                            encoded_out = i[$clog2(WIDTH)-1:0];
                            valid = 1'b1;
                            break;
                        end
                    end
                end
            end
        endcase
    endgenerate

endmodule
```

### Example 8: Parallel Adder Generator
parallel_adder_generator - Generate for loops creating repetitive structures

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_8__parallel_adder_generator/")


### Example 9: Memory Type Selector
memory_type_selector - Generate if-else for conditional compilation

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_9__memory_type_selector/")


### Example 10: Encoder Width Selector
encoder_width_selector - Generate case for parameter-based selection

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_10__encoder_width_selector/")


## Introduction to Interfaces

Interfaces provide a powerful way to group related signals and simplify connections between modules. They help reduce port lists and improve code maintainability.

### Basic Interface Declaration

```systemverilog
interface memory_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16
) (
    input logic clk,
    input logic reset_n
);

    // Interface signals
    logic                    valid;
    logic                    ready;
    logic                    we;
    logic [ADDR_WIDTH-1:0]   addr;
    logic [DATA_WIDTH-1:0]   wdata;
    logic [DATA_WIDTH-1:0]   rdata;
    logic                    error;
    
    // Tasks and functions can be defined in interfaces
    task write_transaction(
        input logic [ADDR_WIDTH-1:0] address,
        input logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b1;
        addr <= address;
        wdata <= data;
        @(posedge clk);
        while (!ready) @(posedge clk);
        valid <= 1'b0;
        we <= 1'b0;
    endtask
    
    task read_transaction(
        input  logic [ADDR_WIDTH-1:0] address,
        output logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b0;
        addr <= address;
        @(posedge clk);
        while (!ready) @(posedge clk);
        data = rdata;
        valid <= 1'b0;
    endtask

endinterface
```

### Using Interfaces in Modules

```systemverilog
// Memory controller module
module memory_controller (
    memory_if.slave  cpu_if,    // CPU interface (slave perspective)
    memory_if.master mem_if     // Memory interface (master perspective)
);

    // Interface connection logic
    always_comb begin
        // Forward CPU requests to memory
        mem_if.valid = cpu_if.valid;
        mem_if.we    = cpu_if.we;
        mem_if.addr  = cpu_if.addr;
        mem_if.wdata = cpu_if.wdata;
        
        // Forward memory responses to CPU
        cpu_if.ready = mem_if.ready;
        cpu_if.rdata = mem_if.rdata;
        cpu_if.error = mem_if.error;
    end

endmodule

// Memory module
module memory (
    memory_if.slave mem_if
);

    localparam int DEPTH = 2**mem_if.ADDR_WIDTH;
    logic [mem_if.DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge mem_if.clk or negedge mem_if.reset_n) begin
        if (!mem_if.reset_n) begin
            mem_if.ready <= 1'b0;
            mem_if.rdata <= '0;
            mem_if.error <= 1'b0;
        end else begin
            mem_if.ready <= mem_if.valid;
            mem_if.error <= 1'b0;
            
            if (mem_if.valid) begin
                if (mem_if.we) begin
                    mem_array[mem_if.addr] <= mem_if.wdata;
                end else begin
                    mem_if.rdata <= mem_array[mem_if.addr];
                end
            end
        end
    end

endmodule
```

### Interface Instantiation and Connection

```systemverilog
module top_level;
    logic clk, reset_n;
    
    // Interface instances
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) cpu_mem_if(clk, reset_n);
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) ctrl_mem_if(clk, reset_n);
    
    // Module instances
    cpu cpu_inst (
        .clk(clk),
        .reset_n(reset_n),
        .mem_if(cpu_mem_if.master)  // CPU is master
    );
    
    memory_controller ctrl_inst (
        .cpu_if(cpu_mem_if.slave),   // Controller is slave to CPU
        .mem_if(ctrl_mem_if.master)  // Controller is master to memory
    );
    
    memory mem_inst (
        .mem_if(ctrl_mem_if.slave)   // Memory is slave
    );

endmodule
```

### Example 11: Basic Memory Interface
basic_memory_interface - Simple interface declaration and usage

In [ ]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_11__basic_memory_interface/")


### Example 12: Basic Memory Interface
interface_with_tasks - Interface with embedded tasks and functions

## Modports and Clocking Blocks

Modports define different views of an interface for different modules, while clocking blocks provide synchronous timing control.

### Modports

Modports specify which signals are inputs, outputs, or inouts from a particular module's perspective.

```systemverilog
interface axi4_lite_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 32
) (
    input logic aclk,
    input logic aresetn
);

    // Write Address Channel
    logic [ADDR_WIDTH-1:0]  awaddr;
    logic [2:0]             awprot;
    logic                   awvalid;
    logic                   awready;
    
    // Write Data Channel
    logic [DATA_WIDTH-1:0]  wdata;
    logic [(DATA_WIDTH/8)-1:0] wstrb;
    logic                   wvalid;
    logic                   wready;
    
    // Write Response Channel
    logic [1:0]             bresp;
    logic                   bvalid;
    logic                   bready;
    
    // Read Address Channel
    logic [ADDR_WIDTH-1:0]  araddr;
    logic [2:0]             arprot;
    logic                   arvalid;
    logic                   arready;
    
    // Read Data Channel
    logic [DATA_WIDTH-1:0]  rdata;
    logic [1:0]             rresp;
    logic                   rvalid;
    logic                   rready;
    
    // Master modport (drives address/data, receives responses)
    modport master (
        input  aclk, aresetn,
        output awaddr, awprot, awvalid,
        input  awready,
        output wdata, wstrb, wvalid,
        input  wready,
        input  bresp, bvalid,
        output bready,
        output araddr, arprot, arvalid,
        input  arready,
        input  rdata, rresp, rvalid,
        output rready
    );
    
    // Slave modport (receives address/data, drives responses)
    modport slave (
        input  aclk, aresetn,
        input  awaddr, awprot, awvalid,
        output awready,
        input  wdata, wstrb, wvalid,
        output wready,
        output bresp, bvalid,
        input  bready,
        input  araddr, arprot, arvalid,
        output arready,
        output rdata, rresp, rvalid,
        input  rready
    );
    
    // Monitor modport (all inputs for verification)
    modport monitor (
        input aclk, aresetn,
        input awaddr, awprot, awvalid, awready,
        input wdata, wstrb, wvalid, wready,
        input bresp, bvalid, bready,
        input araddr, arprot, arvalid, arready,
        input rdata, rresp, rvalid, rready
    );

endinterface
```

### Clocking Blocks

Clocking blocks define synchronous timing relationships and provide a clean way to handle clocked signals in testbenches.

```systemverilog
interface processor_if (
    input logic clk,
    input logic reset_n
);

    logic [31:0] instruction;
    logic [31:0] pc;
    logic        valid;
    logic        ready;
    logic        stall;
    logic        flush;
    
    // Clocking block for testbench use
    clocking cb @(posedge clk);
        default input #1step output #2ns;  // Input skew and output delay
        
        input  pc, valid, ready;
        output instruction, stall, flush;
    endclocking
    
    // Separate clocking block for different timing requirements
    clocking slow_cb @(posedge clk);
        default input #5ns output #10ns;
        
        input  pc, valid;
        output instruction;
    endclocking
    
    // Modports with clocking blocks
    modport tb (
        clocking cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        output pc, valid, ready,
        input  instruction, stall, flush
    );

endinterface
```

### Advanced Clocking Block Example

```systemverilog
interface memory_test_if (
    input logic clk,
    input logic reset_n
);

    logic [15:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic        we;
    logic        re;
    logic        valid;
    logic        ready;
    
    // Clocking block with different timing for different signals
    clocking driver_cb @(posedge clk);
        default input #2ns output #1ns;
        
        output addr, wdata, we, re, valid;
        input  rdata, ready;
    endclocking
    
    // Monitor clocking block samples everything
    clocking monitor_cb @(posedge clk);
        default input #1step;
        
        input addr, wdata, rdata, we, re, valid, ready;
    endclocking
    
    // Synchronous reset clocking block
    clocking reset_cb @(posedge clk);
        input reset_n;
    endclocking
    
    modport driver (
        clocking driver_cb,
        input clk, reset_n
    );
    
    modport monitor (
        clocking monitor_cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        input  addr, wdata, we, re, valid,
        output rdata, ready
    );

endinterface
```

### Using Clocking Blocks in Testbenches

```systemverilog
module memory_testbench;
    logic clk = 0;
    logic reset_n;
    
    always #5ns clk = ~clk;  // 100MHz clock
    
    memory_test_if mem_if(clk, reset_n);
    
    // DUT instantiation
    memory dut (
        .mem_if(mem_if.dut)
    );
    
    // Test program using clocking blocks
    initial begin
        reset_n = 0;
        ##2 reset_n = 1;  // Wait 2 clock cycles
        
        // Write operation using clocking block
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.wdata <= 32'hDEADBEEF;
        mem_if.driver_cb.we    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;  // Wait 1 clock cycle
        
        wait (mem_if.driver_cb.ready);  // Wait for ready
        
        mem_if.driver_cb.we    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##2;  // Wait before read
        
        // Read operation
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.re    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;
        
        wait (mem_if.driver_cb.ready);
        
        $display("Read data: %h", mem_if.driver_cb.rdata);
        
        mem_if.driver_cb.re    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##5;
        $finish;
    end

endmodule
```

axi4_lite_interface - Complete interface with master/slave/monitor modports

clocked_processor_interface - Clocking blocks with different timing requirements

testbench_with_clocking - Using clocking blocks in verification environment

## Summary

This chapter covered the essential concepts of SystemVerilog modules and interfaces:

**Modules** form the basic building blocks with proper port declarations and hierarchical instantiation capabilities.

**Parameters and localparams** enable configurable and reusable designs with type safety and parameter validation.

**Generate blocks** provide powerful compile-time code generation for creating repetitive structures and conditional compilation.

**Interfaces** simplify complex designs by grouping related signals and providing reusable communication protocols.

**Modports** define different perspectives of interfaces for various modules, ensuring proper signal direction and access control.

**Clocking blocks** provide precise timing control for synchronous designs, particularly useful in verification environments.

These features work together to create scalable, maintainable, and reusable SystemVerilog designs that can handle complex digital systems efficiently.